In [1]:
import os
import time
import random
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

import gym
from gym import spaces
from PIL import Image
from collections import deque, namedtuple

from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

/Users/mathemage/anaconda3/envs/tf/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/mathemage/anaconda3/envs/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from environments import Snake
from new_agents import *

In [3]:
env = Snake((8, 8))
#aa = DuelDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5], 
#              save_path="snake_models", model_name="test")
#aa = CatDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5], num_atoms=51,
#                 save_path="snake_models", model_name="test")
#aa = SACAgent(env=env, num_actions=3, temperature=0.05,
#              save_path="snake_models", model_name="test")
aa = GMMDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5],
                   save_path="snake_models", model_name="test3")

In [4]:
aa.set_parameters(max_episode_length=1000, replay_memory_size=100000, replay_start_size=10000,
                  discount_factor=0.999, final_eps=0.01, annealing_steps=100000)

In [ ]:
aa.train(gpu_id=-1, exploration="boltzmann", save_freq=1000000, max_num_epochs=50, tau=1e-3, batch_size=32)

frame count: 5328
average reward: -0.858
epsilon: 0.947
average lifetime: 10.656
-------------------------------
frame count: 12126
average reward: -0.822
epsilon: 0.88
average lifetime: 13.596
-------------------------------
frame count: 22137
average reward: -0.746
epsilon: 0.781
average lifetime: 20.022
-------------------------------
frame count: 35153
average reward: -0.642
epsilon: 0.652
average lifetime: 26.032
-------------------------------


In [6]:
def kl_gaussians(m1, s1, m2, s2):
    m1 = m1[:, :, None]
    m2 = m2[:, None, :]
    s1_2 = tf.square(s1)
    s2_2 = tf.square(s2)
    s1_2 = s1_2[:, :, None]
    s2_2 = s2_2[:, None, :]
    t1 = 0.5 * (tf.log(s2_2) - tf.log(s1_2))
    t2 = (s1_2 + (m2 - m1) ** 2) / (2 * s2_2)
    return t1 + t2 - 0.5

In [30]:
m1 = tf.placeholder(tf.float32, (None, 2))
s1 = tf.placeholder(tf.float32, (None, 2))
m2 = tf.placeholder(tf.float32, (None, 2))
s2 = tf.placeholder(tf.float32, (None, 2))
p1 = tf.placeholder(tf.float32, (None, 2))
p2 = tf.placeholder(tf.float32, (None, 2))

In [8]:
sess = tf.Session()

In [48]:
feed_dict = {m1:np.random.rand(1, 2) * 1e-1, 
             s1:np.random.rand(1, 2) * 1e-1,
             m2:np.random.rand(1, 2) * 1e-1,
             s2:np.random.rand(1, 2) * 1e-1,
             p1:np.array([[0.3, 0.7]]),
             p2:np.array([[0.4, 0.6]])
             }

In [49]:
sess.run(kl_gaussians(m1, s1, m2, s2), feed_dict=feed_dict)

array([[[1.2162083 , 1.2600183 ],
        [0.25347954, 0.30460095]]], dtype=float32)

In [50]:
def kl_mixtures(p1, m1, s1, p2, m2, s2):
    kl1 = kl_gaussians(m1, s1, m1, s1)
    kl2 = kl_gaussians(m1, s1, m2, s2)
    kl1 = tf.exp(-kl1)
    kl2 = tf.exp(-kl2)

    t1 = tf.einsum('abc,ac->ab', kl1, p1)
    t2 = tf.einsum('abc,ac->ab', kl2, p2)
    return tf.reduce_mean(tf.multiply(p1, tf.log(t1) - (tf.log(t2))))

In [51]:
sess.run(kl_mixtures(p1, m1, s1, p2, m2, s2), feed_dict=feed_dict)

0.106065795